In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta



pickle_path=os.path.join('..', 'data', 'flow_counts_preprocessed')
df=pd.read_pickle(pickle_path)

In [2]:
#create datelist
# we remove Saturday and Sundays and Public holidays by checking that at least 25 people are around
weekdays=[0,1,2,3,4] #mon, tue, wed, thu, fri
date_list_all=list(df[(df['time_numeric']==12.33)]['date'])
date_list_weekdays=[date for date in date_list_all if date.weekday() in weekdays]
date_list_weekends_and_holidays=list(df[(df['time_numeric']==12.33)&(df['num_people_inside']<25)]['date'])
date_list_holidays=[date for date in date_list_weekends_and_holidays if date.weekday() in weekdays]
date_list_workdays=list(df[(df['time_numeric']==12.33)&(df['num_people_inside']>25)]['date'])
weekday_date_lists={}
for i in weekdays:
    weekday_date_lists[i]=[date for date in date_list_workdays if date.weekday()==i]


In [6]:
numeric_time=12.33
num_people_by_date=pd.DataFrame(index=date_list_all)
num_people_by_date['num_people_at_12_33']=df.loc[df['time_numeric']==12.33][['num_people_inside','date']].set_index('date')
#num_people_by_date.rename({'num_people_inside':'num_people_inside_0_12_33'}, axis=1)
example_date=date(year=2022, month=5, day=23)
num_people_by_date.loc[example_date][0]
num_people_by_date['num_people_at_10_00']=df.loc[df['time_numeric']==10.00][['num_people_inside','date']].set_index('date')
num_people_by_date['difference_10_and_12']=num_people_by_date.apply(lambda row: row['num_people_at_10_00']-row['num_people_at_12_33'], axis=1)
num_people_by_date['normalized_difference_10_and_12']=num_people_by_date.apply(lambda row: row['difference_10_and_12']/ row['num_people_at_10_00'], axis=1)
labels_by_date=num_people_by_date['num_people_at_12_33']
features_by_date=pd.DataFrame(index=date_list_all, columns=['num_people_0_10_00', '7_days_ago_10_00', '7_days_ago_norm_dif'])
features_by_date['num_people_0_10_00']= num_people_by_date['num_people_at_10_00']

for day in date_list_all:
    time_delta=timedelta(days=7)
    try:
        features_by_date[day,'7_days_ago_10_00']=num_people_by_date.loc[day-time_delta,'num_people_at_10_00']
    except KeyError:
        features_by_date[day,'7_days_ago_10_00']=float('nan')
features_by_date.iloc(example_date)

C:\Users\mefl\AppData\Local\Temp\ipykernel_11736\735385191.py:9: RuntimeWarning: invalid value encountered in longlong_scalars
  num_people_by_date['normalized_difference_10_and_12']=num_people_by_date.apply(lambda row: row['difference_10_and_12']/ row['num_people_at_10_00'], axis=1)
C:\Users\mefl\AppData\Local\Temp\ipykernel_11736\735385191.py:9: RuntimeWarning: divide by zero encountered in longlong_scalars
  num_people_by_date['normalized_difference_10_and_12']=num_people_by_date.apply(lambda row: row['difference_10_and_12']/ row['num_people_at_10_00'], axis=1)
C:\Users\mefl\AppData\Local\Temp\ipykernel_11736\735385191.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_by_date[day,'7_days_ago_10_00']=num_people_by_date.loc[day-time_delta,'num_

ValueError: No axis named 2022-05-23 for object type DataFrame

In [ ]:
features_by_date.loc[example_date,'num_people_7_10_00']

In [5]:
type(float('nan'))

float

AttributeError: 'str' object has no attribute 'append'

In [ ]:
ex_frame=pd.DataFrame({'a':[0,3,4], 'b':[7,4,4]})
ex_frame.rename({'a':'c', 'b':'e'}, axis=1)

In [ ]:
{'a':1,'b':2}

In [ ]:
for group, frame in df.groupby('date'):
    if group==date(year=2022,month=3,day=5):
        example_frame=frame
#example_frame[example_frame['time_numeric']==12.33]['num_people_inside'].iloc[0]


def get_function_num_people_inside_at(numeric_time):
    def func(dataframe):
        print(dataframe)
        return dataframe[dataframe['time_numeric']==numeric_time]['num_people_inside'].iloc[0]
    return func

# test function
func=get_function_num_people_inside_at(10.00)
assert func(example_frame) == 0

df.groupby('date').aggregate(func=get_function_num_people_inside_at(10.00))

In [ ]:
#create a dataframe for monday

In [ ]:
df_monday=df[(df['time_numeric']==12.33)&(df['num_people_inside']>20)&(df['weekday']=='Monday')][['date', 'zurich_datetime', 'num_people_inside', 'weekday']].copy

In [ ]:
ex_date=date(year=2022,month=9,day=23)
num_past_days_to_consider=4
[ex_date- timedelta(days=7*i) for i in list(range(1,num_past_days_to_consider+1))]
df[df['date'].isin([ex_date- timedelta(days=7*i) for i in list(range(1,num_past_days_to_consider+1))])]

In [ ]:
#create date list


In [ ]:
df.groupby(['weekday','time'])['num_people_inside'].aggregate([np.mean, np.std])

In [ ]:
df[df['weekday'].isin(['Monday'])].groupby('time')['num_people_inside'].aggregate([np.mean, np.std])

In [ ]:
weekday='Monday'
mean_values=df[df['weekday'].isin([weekday])].groupby('time')['num_people_inside'].aggregate([np.mean])
std_values=df[df['weekday'].isin([weekday])].groupby('time')['num_people_inside'].aggregate([np.std])
quantile_90=df[df['weekday'].isin([weekday])].groupby('time')['num_people_inside'].aggregate(np.percentile,q=90)
quantile_10=df[df['weekday'].isin([weekday])].groupby('time')['num_people_inside'].aggregate(np.percentile,q=10)
quantile_75=df[df['weekday'].isin([weekday])].groupby('time')['num_people_inside'].aggregate(np.percentile,q=75)
quantile_25=df[df['weekday'].isin([weekday])].groupby('time')['num_people_inside'].aggregate(np.percentile,q=25)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(mean_values)
plt.plot(quantile_90)
plt.plot(quantile_10)
plt.plot(quantile_75)
plt.plot(quantile_25)
plt.legend(['mean_values', 'quantile_90', 'quantile_10', 'quantile_75', 'quantile_25'])

In [ ]:
std_values

In [ ]:
mean_values

In [ ]:
type(mean_values['mean'])